In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 1. 파일로더

- PyPDFLoader를 사용하여 pdf 파일을 로드하였다.
- 보험 약관 가이드를 만들고자 하는 목적
- 약관 상품 설명서 pdf 파일을 다운받아 파일을 불러왔다.

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('./data/메리츠보험.pdf')

pdf_docs = pdf_loader.load()
print(f'PDF 문서: {len(pdf_docs)}')

c:\project\llm_edu\001_chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PDF 문서: 1056


* 약관의 길이가 길었다. pdf 문서의 페이지는 1056 페이지.

# 2. 텍스트 스플리터
- RecursiveCharacterTextSplitter 사용
- chunk_size=700, chunk_overlap=100, separators=["\n\n","\n"]
- 약관의 경우, 청크 사이즈가 작으면 좋지 않다는 gpt의 의견에 청크사이즈를 크게 설정.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=100,
    separators=["\n\n","\n", " "]
)

chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

생성된 텍스트 청크 수: 2528
각 청크의 길이: [69, 5, 694, 279, 5, 699, 294, 5, 693, 158, 5, 698, 452, 5, 698, 404, 5, 695, 314, 5, 676, 252, 6, 696, 140, 6, 695, 403, 553, 6, 133, 588, 414, 193, 562, 272, 259, 401, 577, 612, 576, 697, 463, 263, 194, 519, 218, 579, 330, 657, 633, 252, 6, 664, 620, 628, 693, 547, 6, 660, 674, 664, 677, 407, 6, 691, 684, 695, 196, 6, 685, 619, 691, 304, 6, 685, 697, 694, 143, 6, 660, 698, 659, 389, 6, 687, 666, 621, 457, 6, 664, 653, 629, 627, 542, 562, 6, 668, 652, 658, 696, 429, 6, 645, 623, 694, 627, 683, 632, 6, 668, 633, 596, 641, 243, 6, 632, 681, 696, 699, 672, 545, 6, 619, 629, 644, 625, 327, 6, 646, 695, 666, 623, 697, 119, 6, 663, 616, 620, 679, 610, 449, 6, 433, 542, 623, 697, 586, 6, 629, 628, 678, 655, 650, 479, 6, 669, 698, 699, 484, 6, 643, 655, 602, 657, 607, 6, 693, 223, 502, 500, 128, 6, 697, 254, 606, 6, 698, 346, 6, 698, 408, 6, 697, 461, 6, 698, 447, 628, 504, 229, 29, 134, 628, 274, 473, 678, 6, 697, 250, 6, 699, 423, 6, 697, 353, 6, 697, 394, 6, 69

* 생성된 텍스트 청크 수는 2528. 
* 처음 실습 시에는 " " 조건을 빼고 주었고(chunk_size=1000), 1000여개의 청크가 생성되었었다. 각 청크의 길이가 2000자가 넘는 청크도 존재하여서, 재시도 시, 조건을 추가하였다.

# 3. 임베딩 모델
- Ollama 임베딩 모델을 사용
- 실습시 Ollama의 bge-m3 모델 적용(벡터 저장 50분 소요)
- 재시도 시, nomic-embed-text 모델 적용
- 답변 성능 저하고, 다시 bge-m3 모델 사용

In [4]:
# Ollama 임베딩 모델을 사용한 FAISS 벡터 저장소 생성
import faiss 
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_ollamas = OllamaEmbeddings(model="bge-m3")


# 4. 벡터 저장소 생성 및 저장
- FAISS 벡터 저장소 사용
- 유클리드 거리 사용
- 임베딩 차원 768 설정

In [5]:
# FAISS 인덱스 초기화 (유클리드 거리 사용)
dim = 1024  # 임베딩 차원
faiss_index = faiss.IndexFlatL2(dim)  

# FAISS 벡터 저장소 생성
faiss_db = FAISS(
    embedding_function=embeddings_ollamas,
    index = faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

# 저장된 문서의 갯수 확인
print(faiss_db.index.ntotal)

0


In [6]:
from langchain_core.documents import Document
import uuid

documents = [
    (
        chunk.page_content,
        f"{chunk.metadata['source']} - {chunk.metadata['page']}페이지"
    )
    for chunk in chunks
]

# Document 객체 생성
doc_objects = []
for content, source in documents:
    doc = Document(
        page_content=content,
        metadata={"source": source},
    )
    doc_objects.append(doc)

# 문서 id 생성
doc_ids = [str(uuid.uuid4()) for _ in range(len(chunks))]

# 문서를 벡터 저장소에 저장
# 힌트: faiss_db.add_documents(chunks, ids=doc_ids) 사용
added_doc_ids = faiss_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

2528개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['f3ae8be9-9ae8-45a9-b281-05f17c2a2630', '106ac4ba-44eb-4afb-add6-ac7b20048f6a', '28c433c4-d479-430e-b7a4-aa7242e92139', '8089d74e-5c68-44c3-84ab-fea05b03b793', '8c4dc7fd-70f4-45ea-a7b2-e7582c140af6', '1035a70f-cb3c-4d60-a69b-1d2864f553e9', '5a6e0936-5d43-45dd-96b4-7c47dc338715', '0721f9fd-8f9c-40e0-8ff0-1d45ba9db1b9', '6810bb5a-41e5-46ac-9512-f540046b2855', '3612f456-4bad-4fac-8ba7-0de4ce876688', '50b4e0db-694d-4a7c-bc14-1d931688a68f', '7861d7c5-15de-42ac-897e-f630417c10fb', '649addd3-cb3b-467c-98bf-730890eac508', '6d5fb57e-4497-424d-a1f1-0a1dc54bdaa3', '7399a9a8-456e-4f3c-9f36-2e50cfbb13cd', '9cbf9aa6-f487-489b-9520-bc8568a1a53a', '90ae7c33-815d-4d1b-9993-56b4c62e434d', 'fcf25ddd-fb48-4e61-8eb7-2d4eccb7df0a', 'cb9aec0b-6537-48cc-aed7-571d77eefa69', 'bfed8493-533c-453b-a205-6a212caa677b', '498107d6-d66f-4af5-a821-7b56866bf6d4', '70992773-c083-4703-adf6-d8eafdede6c6', '03062317-7970-4df7-a2cd-a7dbef2dd930', '262081f9-c12f-4051-8cfc-0d222c7f6a87', '4e30

`bge-m3` 
- chunk_size: 1000
- 1363개의 문서가 생성.
- 시간 50분 소요. 
- 대부분의 질문에 답변이 가능했다.

`nomic-embed-text`
- chunk_size: 700, separators " " 추가
- 2528개 문서 생성. 
- 시간 7분 50초 소요
- 그러나 답변 성능 저하. 거의 모든 질문에서 컨텍스트를 찾지 못하였다.

`bge-m3` 
- chunk_size: 700
- 2528개의 문서가 생성.
- 시간 47분 소요. 
- 대부분의 질문에 답변이 가능했다.

# 5. 검색기 정의
- mmr 검색기를 생성하였다. 
- 상위 3개 문서를 검색하는 Retriever를 사용하였다. 

In [7]:
# mmr 검색기 생성
# 힌트: faiss_db.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 10, 'lambda_mult': 0.3})
# lambda_mult를 낮게 설정하여 다양성을 높임
faiss_mmr_retriever = faiss_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k':3,
        'fetch_k':8,
        'lambda_mult':0.5,
    }
)

# 검색 테스트 
query = "암 진단금은 얼마인가?"
# 힌트: faiss_mmr_retriever.invoke(query) 사용
retrieved_docs = faiss_mmr_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]}")
    print("-" * 100)

쿼리: 암 진단금은 얼마인가?
검색 결과:
-1-
- 224 -
19. 암진단비(유사암제외)보장 특별약관제1조(보험금의 지급사유)회사는 보험증권에 기재된 피보험자가 이 특별약관의 보험기간 중 암보장개시일 이후에「암(유사암제외)」...만 해당)  < 암(유사암 및 소액암 제외) >
    90일보장제외보험가입금액의 100% 2021.4.10.   계약일    2021.7.9  암보장개시일      < 소액암 >
----------------------------------------------------------------------------------------------------
-2-
□ 보험금 지급관련 특히 유의할 사항 ○ 암 관련 보장   - 계약일부터 90일 이내에 암으로 진단받은 경우에는 보험금을 지급하지 않습니다.   - 90일이 경과한 이후에도 암 진.... ○ 배상책임 관련 보장 등 다수계약의 비례보상에 관한 사항   - 이 계약에서 보장하는 위험과 같은 위험을 보장하는 다른 계약(공제계약 포함)이 있을 경우에는 각 계약에 대하여
----------------------------------------------------------------------------------------------------
-3-
하지 않으며,「소액암」으로 진단확정 후「암(유사암 및 소액암 제외)」로 진단확정시에는 암진단비(유사암 및 소액암 제외)진단시에 해당하는 보험금을 지급하지 않습니다.피보험자가 암보...44(기타 피부의 악성신생물) 및 C73(갑상선의 악성신생물)을 제외한 질병을 말합니다. 또한, 전암(前癌)상태(암으로 변하기 이전 상태. Premalignant condition
----------------------------------------------------------------------------------------------------


# 6. RAG 프롬프트 구성
- 작성 기준을 토대로 프롬프트를 구성하였다.
- ChatPromptTemplate을 사용

In [8]:
# Prompt 템플릿 (여기에 작성하세요)
from langchain_core.prompts import ChatPromptTemplate

template = """
당신은 보험 약관 분석 도우미 입니다

아래 제공된 컨텍스트에 기반하여 질문에 답변하세요.

[작업 지침]
- 반드시 컨텍스트에 포함된 정보만 사용하십시오.
- 외부 지식이나 추측을 사용하지 마십시오.
- 컨텍스트에서 근거를 찾을 수 없으면 "컨텍스트에서 확인할 수 없습니다."라고 답하십시오.
- 답변은 사실에 기반해야 합니다.
- 답변은 한글로 작성되어야 합니다.

[컨텍스트]
{context}

[질문]
{question}

[답변 형식]
1. 핵심 답변:
2. 근거:
3. 추가 설명 (필요한 경우에만 작성):
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿 출력
prompt.pretty_print()

================================ Human Message =================================


당신은 보험 약관 분석 도우미 입니다

아래 제공된 컨텍스트에 기반하여 질문에 답변하세요.

[작업 지침]
- 반드시 컨텍스트에 포함된 정보만 사용하십시오.
- 외부 지식이나 추측을 사용하지 마십시오.
- 컨텍스트에서 근거를 찾을 수 없으면 "컨텍스트에서 확인할 수 없습니다."라고 답하십시오.
- 답변은 사실에 기반해야 합니다.
- 답변은 한글로 작성되어야 합니다.

[컨텍스트]
{context}

[질문]
{question}

[답변 형식]
1. 핵심 답변:
2. 근거:
3. 추가 설명 (필요한 경우에만 작성):



`RAG 체인 구성`
- ChatOpenAI 사용 ('gpt-4o-mini' 모델)
- temperature: 답변의 일관성을 가져가도록 0으로 설정

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
# 힌트: ChatOpenAI(model='gpt-4o-mini', temperature=0) 사용
llm = ChatOpenAI(
    model='gpt-4o-mini', 
    temperature=0
)

# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

# RAG 체인 생성
# 힌트: {'context': faiss_mmr_retriever | format_docs, 'question': RunnablePassthrough()} | prompt | llm | StrOutputParser()
rag_chain = {'context': faiss_mmr_retriever | format_docs, 'question': RunnablePassthrough()} | prompt | llm | StrOutputParser()

# 체인 실행
query = "암 진단시 보장금은 어떻게 되나요?"
output = rag_chain.invoke(query)

print(f"쿼리: {query}")
print("답변:")
print(output)

쿼리: 암 진단시 보장금은 어떻게 되나요?
답변:
1. 핵심 답변: 암 진단 시 보장금은 진단된 암의 종류에 따라 다르며, 유사암의 경우 최초 1회의 진단확정에 한하여 지급되고, 암(유사암 및 소액암 제외)의 경우에는 보험가입금액의 100%가 지급됩니다. 

2. 근거: 
   - "「유사암」은 각각 최초 1회의 진단확정에 한하여 지급하며,「유사암」이 하나의 질병에 의해 각각 진단확정된 경우에는 하나의 진단비를 지급합니다."
   - "피보험자가 보장개시일 이후에 사망하고 그 후에「암(유사암 제외)」로 사망한 사실이 확인된 경우에는 제1항의 암진단비(유사암 제외)를 지급합니다."

3. 추가 설명: 암 진단 후 지급되는 보험금은 계약일로부터 90일 이내에 진단받은 경우 지급되지 않으며, 90일이 경과한 후에도 계약일부터 일정 기간 이내에 진단받은 경우 보험금이 삭감될 수 있습니다.


`Gradio 스트리밍 구현`
- ChatInterface 사용

In [10]:
import gradio as gr
from typing import Iterator

# 스트리밍 응답 생성 함수
def get_streaming_response(message: str, history) -> Iterator[str]:
    
    # RAG Chain 실행 및 스트리밍 응답 생성
    response = ""
    for chunk in rag_chain.stream(message):
        if isinstance(chunk, str):
            response += chunk
            yield response

# Gradio 인터페이스 설정
# 힌트: gr.ChatInterface(fn=get_streaming_response, title="RAG 기반 질의응답 시스템", description="...", examples=[...])
demo = gr.ChatInterface(
    fn=get_streaming_response,
    title="보험 분석 어시스턴스"
)

# 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
